In [ ]:
%reset -f
%load_ext autoreload
%autoreload 2

%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np

from stellarwinds.magnetogram import reader_writer
from stellarwinds.magnetogram import converter
from stellarwinds.magnetogram import rotate
from stellarwinds.magnetogram import plots
from stellarwinds.magnetogram import plot_pfss
from stellarwinds.magnetogram import plot_zdi
from stellarwinds.magnetogram import coefficients
from stellarwinds.magnetogram import zdi_magnetogram
from stellarwinds.magnetogram import geometry
from stellarwinds.magnetogram import pfss_magnetogram

# Building a magnetogram from scratch

In [ ]:
coeffs = coefficients.Coefficients()

# Start with a rotated dipole
coeffs.append(1, 0, 5.0)
coeffs = rotate.rotate_magnetogram_euler_zyz_deg(coeffs, (0, 30, 0))
coeffs = converter.map_to_positive_orders(coeffs)

# Add some noise
noise = coefficients.noise(degree_max=15, beta=1) * 100
coeffs += noise

# Plot
coeffs = coeffs.scale(converter.forward_conversion_factor, 1)
fig, axs = plt.subplots(1,3, figsize=(18, 5))
plot_pfss.plot_components(coeffs, axs=axs)

# Rotating a magnetogram
The PFSS magnetogram is entirely determined by the radial coefficients, so it should not be surprising that the polar and azimuthal coefficients change when the radial coefficients are rotated.

In [ ]:
file = "/Users/u1092841/Documents/PHD/toupies-magnetograms-colin/coeff-TYC6349-0200-1.dat"
coeffs = reader_writer.read_magnetogram_file(file)
alpha, beta, gamma = coefficients.hsplit(coeffs)
coeffs = alpha.scale(converter.forward_conversion_factor, 1)

fig, axs = plt.subplots(1,3, figsize=(18, 5))
plot_pfss.plot_components(coeffs, axs=axs)
plt.show()

In [ ]:
rotated = rotate.rotate_magnetogram_euler_zyz_deg(coeffs, (0, 270, 0))
rotated = converter.map_to_positive_orders(rotated)
fig, axs = plt.subplots(1,3, figsize=(18, 5))
plot_pfss.plot_components(rotated, axs=axs)
plt.show()

Notice how while the energies of each harmonics $(\ell, m)$ are changed by the rotation, the total energy at each degree $\ell$ is as before.

In [ ]:
zc = zdi_magnetogram.from_coefficients(coeffs)
fig, axs = plt.subplots(1,2, figsize=(12,5))
plot_zdi._plot_energy(zc, ax=axs[1])
plot_zdi.plot_energy_by_degree(zc, ax=axs[0])

zc = zdi_magnetogram.from_coefficients(rotated)
fig, axs = plt.subplots(1,2, figsize=(12,5))
plot_zdi._plot_energy(zc, ax=axs[1])
plot_zdi.plot_energy_by_degree(zc, ax=axs[0]);

## Load an image 
For fun, read in an image and create a magnetogram from it.

In [ ]:
import urllib.request
import IPython.display
import PIL.Image
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logging.debug("test")
from stellarwinds.magnetogram import image_converter
image_converter.log.setLevel(logging.INFO)

url = 'https://svs.gsfc.nasa.gov/vis/a000000/a003400/a003487/landmask4K.png'
file, head = urllib.request.urlretrieve(url)

In [ ]:
full_image = PIL.Image.open(file)
IPython.display.display(full_image)
print(f'The image is {full_image.size} pixels in size.')

The image is way too big so resize it to $4\ell_\text{max}$ in the longitude direction.

In [ ]:
l_max = 30
resize_factor = full_image.size[0] // (4 * l_max)
new_size = [d // resize_factor for d in full_image.size]
image = full_image.resize(new_size, PIL.Image.LANCZOS)
IPython.display.display(image)
print(f'The resized image is {image.size} pixels in size.')

In [ ]:
image_in = np.array(image)
while len(image_in.shape) > 2:
    print(image_in.shape)
    image_in = np.average(image_in, axis=2)
image_in = image_in.T - np.average(image_in)


coeffs, zg = image_converter.from_image(image_in, lmax=l_max)
image_out = np.real(image_converter.to_image(coeffs, zg))

vmax = np.maximum(np.max(image_in), np.max(image_out))
vmin = np.minimum(np.min(image_in), np.min(image_out))
v_max_abs = np.maximum(np.abs(vmax), np.abs(vmin))

fig, axs = plt.subplots(2, 1, figsize=(8, 8))

for data, ax in zip((image_in, image_out), axs):
    img = plots.plot_equirectangular(zg, np.real(data), ax)

cb = fig.colorbar(img, ax=axs.ravel().tolist())

In [ ]:
# A ZDI magnetogram usually does not have negative orders, so 
# transform the coefficients so that the negative orders are all zero.
# This changes the imaginary component of the magnetogram, which are
# anyhow ignored in ZDI.
coeffs = converter.map_to_positive_orders(coeffs)

# Create the ZDI magnetogram 
lz = zdi_magnetogram.from_coefficients(coeffs)
polar, azimuth = zg.corners()

# This only returns the real component since it is a ZDI magnetogram.
B = lz.get_radial_field(*zg.centers())

Bscale = np.max(np.abs(B))

fig, ax = plt.subplots(figsize=(8, 3))
img1 = plots.plot_equirectangular(zg, B,
                                  ax,
                                  vmin=-v_max_abs,
                                  vmax=v_max_abs,
                                 )

cb = fig.colorbar(img1, ax=ax)